In [ ]:
!wsl -d <Distributive-Name>
!kubectl create namespace spark
!kubectl apply -f ./docker/kubernetes/deprecated/no-auth-spark-master.yaml -n spark
!kubectl apply -f ./docker/kubernetes/deprecated/no-auth-spark-master.yaml -n spark

# Into everyone kubernetes spark container
!pip install flask aiohttp requests

# Code for worker deploying

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)
"deprecated"

def sum_fibonacci(n: int) -> int:
    total = 0
    for _ in range(n):
        total += 1
    return total

@app.route("/fib_sum", methods=["GET"])
def fib_sum():
    n_str = request.args.get("n", None)
    if n_str is None:
        return jsonify({"error": "Не задан параметр 'n'"}), 400
    try:
        n = int(n_str)
        if n < 0:
            raise ValueError
    except ValueError:
        return jsonify({"error": "Параметр 'n' должен быть целым числом >= 0"}), 400
    result = sum_fibonacci(n)
    response = {
        "n": n,
        "fib_sum": result,
        "note": f"Сумма первых {n} чисел Фибоначчи (F₁…Fₙ)"
    }
    return jsonify(response), 200

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=11111)


# Code for master deploying

In [ ]:
import asyncio
import aiohttp
from time import time_ns

async def get_fib_sum_async(n: int, base_url: str = "http://spark-worker:11111") -> int:
    if n < 0:
        raise ValueError("n должен быть целым числом >= 0")
    url = f"{base_url.rstrip('/')}/fib_sum"
    params = {"n": n}
    timeout = aiohttp.ClientTimeout(total=60)
    async with aiohttp.ClientSession(timeout=timeout) as session:
        try:
            async with session.get(url, params=params) as resp:
                if resp.status != 200:
                    text = await resp.text()
                    raise ValueError(f"Сервер вернул статус {resp.status}: {text}")
                data = await resp.json()
        except aiohttp.ClientError as e:
            raise e from e
    if "fib_sum" not in data:
        raise ValueError(f"Непредвиденный ответ сервера: {data}")
    return data["fib_sum"]


# Пример использования:
async def run_q(n):
    try:
        start = time_ns()
        result = await get_fib_sum_async(n)
        print(f"{(time_ns() - start) / 1_000_000_000}: Сумма первых {n} чисел Фибоначчи: {result}")
    except ValueError as ve:
        print(f"Ошибка валидации или ответ сервера: {ve}")
    except aiohttp.ClientError as ce:
        print(f"Сетевая ошибка при запросе: {ce}")

async def main(nums):
    tasks = [asyncio.create_task(run_q(i)) for i in nums]
    await asyncio.wait(tasks)

# asyncio.run(main(10))
asyncio.run(main([100_000_000, 100_000_000, 100_000_000, 100_000_000, 100_000_000]))

In [2]:
import requests
requests.get("http://mongodb-svc1.default.svc.cluster.local:28017")

ConnectionError: HTTPConnectionPool(host='mongodb-svc1.default.svc.cluster.local', port=28017): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7fe266e78470>: Failed to resolve 'mongodb-svc1.default.svc.cluster.local' ([Errno -3] Temporary failure in name resolution)"))